# Preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

np.random.seed(123)

## Load data

In [2]:
datasets_indexes = range(0, 32)

dfs = {i: pd.read_csv(f'../data/raw/{i}.csv') for i in datasets_indexes}

## Individual changes

### Helpers

In [3]:
def extract_datetime_features(df, datetime_col):
    df_copy = df.copy()
    
    if not pd.api.types.is_datetime64_any_dtype(df_copy[datetime_col]):
        df_copy[datetime_col] = pd.to_datetime(df_copy[datetime_col], errors='coerce')
    
    df_copy[f'{datetime_col}_year'] = df_copy[datetime_col].dt.year
    df_copy[f'{datetime_col}_month'] = df_copy[datetime_col].dt.month
    df_copy[f'{datetime_col}_day'] = df_copy[datetime_col].dt.day
    df_copy[f'{datetime_col}_hour'] = df_copy[datetime_col].dt.hour
    df_copy[f'{datetime_col}_dayofweek'] = df_copy[datetime_col].dt.dayofweek
    df_copy[f'{datetime_col}_quarter'] = df_copy[datetime_col].dt.quarter
    df_copy[f'{datetime_col}_is_weekend'] = df_copy[datetime_col].dt.dayofweek >= 5
    
    df_copy[f'{datetime_col}_month_sin'] = np.sin(2 * np.pi * df_copy[datetime_col].dt.month / 12)
    df_copy[f'{datetime_col}_month_cos'] = np.cos(2 * np.pi * df_copy[datetime_col].dt.month / 12)

    df_copy[f'{datetime_col}_dayofweek_sin'] = np.sin(2 * np.pi * df_copy[datetime_col].dt.dayofweek / 7)
    df_copy[f'{datetime_col}_dayofweek_cos'] = np.cos(2 * np.pi * df_copy[datetime_col].dt.dayofweek / 7)

    df_copy[f'{datetime_col}_hour_sin'] = np.sin(2 * np.pi * df_copy[datetime_col].dt.hour / 24)
    df_copy[f'{datetime_col}_hour_cos'] = np.cos(2 * np.pi * df_copy[datetime_col].dt.hour / 24)
    
    df_copy = df_copy.drop(columns=[datetime_col])
    
    return df_copy

def extract_date_features(df, date_col):
    df_copy = df.copy()
    
    if not pd.api.types.is_datetime64_any_dtype(df_copy[date_col]):
        df_copy[date_col] = pd.to_datetime(df_copy[date_col], errors='coerce')
    
    df_copy[f'{date_col}_year'] = df_copy[date_col].dt.year
    df_copy[f'{date_col}_month'] = df_copy[date_col].dt.month
    df_copy[f'{date_col}_day'] = df_copy[date_col].dt.day
    df_copy[f'{date_col}_dayofweek'] = df_copy[date_col].dt.dayofweek
    df_copy[f'{date_col}_quarter'] = df_copy[date_col].dt.quarter
    df_copy[f'{date_col}_is_weekend'] = df_copy[date_col].dt.dayofweek >= 5

    df_copy[f'{date_col}_month_sin'] = np.sin(2 * np.pi * df_copy[date_col].dt.month / 12)
    df_copy[f'{date_col}_month_cos'] = np.cos(2 * np.pi * df_copy[date_col].dt.month / 12)
    
    df_copy[f'{date_col}_dayofweek_sin'] = np.sin(2 * np.pi * df_copy[date_col].dt.dayofweek / 7)
    df_copy[f'{date_col}_dayofweek_cos'] = np.cos(2 * np.pi * df_copy[date_col].dt.dayofweek / 7)
    
    df_copy = df_copy.drop(columns=[date_col])
    
    return df_copy

def extract_hour_features(df, hour_col):
    df_copy = df.copy()
    
    df_copy[f'{hour_col}_sin'] = np.sin(2 * np.pi * df_copy[hour_col] / 24)
    df_copy[f'{hour_col}_cos'] = np.cos(2 * np.pi * df_copy[hour_col] / 24)
    
    return df_copy

### df0
- remove extra columns
- replace `?` with `NaN`

In [4]:
dfs[0] = dfs[0].drop(columns=["state", "county", "community", "communityname", "fold"])
dfs[0] = dfs[0].replace("?", np.nan).astype(float)
dfs[0].head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.02,0.12,0.45,NaN,NaN,NaN,NaN,0.00,NaN,0.67
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.01,0.21,0.02,NaN,NaN,NaN,NaN,0.00,NaN,0.43
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.02,0.39,0.28,NaN,NaN,NaN,NaN,0.00,NaN,0.12
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.04,0.09,0.02,NaN,NaN,NaN,NaN,0.00,NaN,0.03


### df2
- expand date informations

In [5]:
dfs[2] = dfs[2].drop(columns=["season", "yr", "mnth", "weekday"])
dfs[2] = extract_date_features(dfs[2], "dteday")
dfs[2] = extract_hour_features(dfs[2], "hr")
dfs[2]["cnt"] = dfs[2].pop("cnt") # because extracting new functions puts them at the end
dfs[2].head()

,instant,hr,holiday,workingday,weathersit,temp,atemp,hum,windspeed,casual,...,dteday_dayofweek,dteday_quarter,dteday_is_weekend,dteday_month_sin,dteday_month_cos,dteday_dayofweek_sin,dteday_dayofweek_cos,hr_sin,hr_cos,cnt
0,1,0,0,0,1,0.24,0.2879,0.81,0.0,3,...,5,1,True,0.5,0.866025,-0.974928,-0.222521,0.000000,1.000000,16
1,2,1,0,0,1,0.22,0.2727,0.80,0.0,8,...,5,1,True,0.5,0.866025,-0.974928,-0.222521,0.258819,0.965926,40
2,3,2,0,0,1,0.22,0.2727,0.80,0.0,5,...,5,1,True,0.5,0.866025,-0.974928,-0.222521,0.500000,0.866025,32
3,4,3,0,0,1,0.24,0.2879,0.75,0.0,3,...,5,1,True,0.5,0.866025,-0.974928,-0.222521,0.707107,0.707107,13
4,5,4,0,0,1,0.24,0.2879,0.75,0.0,0,...,5,1,True,0.5,0.866025,-0.974928,-0.222521,0.866025,0.500000,1


### df4
- remove extra target column

In [6]:
dfs[4] = dfs[4].drop(columns=["aveOralF"])
dfs[4].head()

,Gender,Age,Ethnicity,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,...,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1,aveOralM
0,Male,41-50,White,24.0,28.0,0.8,0.7025,35.0300,35.3775,34.4000,...,33.4775,33.3725,33.4925,33.0025,34.5300,34.0075,35.6925,35.6350,35.6525,36.59
1,Female,31-40,Black or African-American,24.0,26.0,0.8,0.7800,34.5500,34.5200,33.9300,...,34.0550,33.6775,33.9700,34.0025,34.6825,34.6600,35.1750,35.0925,35.1075,37.19
2,Female,21-30,White,24.0,26.0,0.8,0.8625,35.6525,35.5175,34.2775,...,34.8275,34.6475,34.8200,34.6700,35.3450,35.2225,35.9125,35.8600,35.8850,37.34
3,Female,21-30,Black or African-American,24.0,27.0,0.8,0.9300,35.2225,35.6125,34.3850,...,34.4225,34.6550,34.3025,34.9175,35.6025,35.3150,35.7200,34.9650,34.9825,37.09
4,Male,18-20,White,24.0,27.0,0.8,0.8950,35.5450,35.6650,34.9100,...,35.1600,34.3975,34.6700,33.8275,35.4175,35.3725,35.8950,35.5875,35.6175,37.04


### df6
- expand  date informations
- move the target column to the last place
- pick random 10% of the records (due to file size)

In [7]:
dfs[6] = extract_datetime_features(dfs[6], "date")
dfs[6]["Appliances"] = dfs[6].pop("Appliances")
dfs[6] = dfs[6].sample(frac=0.1)
dfs[6].head()

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,date_dayofweek,date_quarter,date_is_weekend,date_month_sin,date_month_cos,date_dayofweek_sin,date_dayofweek_cos,date_hour_sin,date_hour_cos,Appliances
9258,20,21.500000,41.59,18.600000,46.463333,20.79,38.730,21.100000,37.290000,18.79,...,2,1,False,1.0,6.123234e-17,0.974928,-0.222521,0.000000,1.000000,70
10439,0,21.500000,38.70,18.856667,42.000000,22.29,39.060,20.823333,37.126667,19.50,...,3,1,False,1.0,6.123234e-17,0.433884,-0.900969,0.866025,0.500000,50
16471,0,22.790000,34.56,20.260000,36.400000,23.50,36.925,22.700000,32.900000,20.89,...,3,2,False,0.5,-8.660254e-01,0.433884,-0.900969,0.500000,0.866025,50
7718,0,20.000000,37.29,17.200000,40.590000,20.89,38.000,18.500000,36.500000,18.29,...,5,1,True,1.0,6.123234e-17,-0.974928,-0.222521,0.965926,-0.258819,50
19622,20,24.426667,44.53,23.318571,43.564286,26.70,38.760,24.760000,43.260000,22.60,...,3,2,False,0.5,-8.660254e-01,0.433884,-0.900969,-0.258819,0.965926,60


### df7
- drop extra target columns
- replace `yes` and `no` with boolean values

In [8]:
dfs[7] = dfs[7].drop(columns=["G1", "G2"])
dfs[7] = dfs[7].replace({"yes": True, "no": False})
dfs[7].head()

C:\Users\jrsh4\AppData\Local\Temp\ipykernel_11632\1226513948.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfs[7] = dfs[7].replace({"yes": True, "no": False})


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,False,False,4,3,4,1,1,3,4,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,True,False,5,3,3,1,1,3,2,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,True,False,4,3,2,2,3,3,6,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,True,True,3,2,2,1,1,5,0,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,False,False,4,3,2,1,2,5,0,13


### df10
- remove extra target column

In [9]:
dfs[10] = dfs[10].drop(columns=["Y2"])
dfs[10].head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84


### df13
- pick random 10% of the records (due to file size)

In [ ]:
dfs[13] = dfs[13].sample(frac=0.1)
dfs[13].head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
27781,5,5,4,6,3,5,4,3,6,6,...,3,8,7,6,6,5,2,2,4,0.470
47625,2,7,5,2,0,6,6,9,10,4,...,4,3,3,4,5,6,3,3,3,0.445
14294,7,2,5,4,6,10,8,5,3,5,...,1,7,3,3,7,5,2,6,5,0.480
40072,5,7,8,3,6,3,7,3,3,6,...,4,4,5,4,7,5,9,5,6,0.510
39350,4,6,5,4,7,7,5,5,3,4,...,4,4,12,1,3,5,5,5,2,0.490


### df15
- expand  date informations
- change `holiday` column to boolean
- pick random 10% of the records (due to file size)


In [ ]:
dfs[15] = extract_datetime_features(dfs[15], 'date_time')
dfs[15]['holiday'] = dfs[15]['holiday'].apply(lambda x: 0 if pd.isna(x) else 1)
dfs[15]["traffic_volume"] = dfs[15].pop("traffic_volume") # because extracting new functions puts them at the end
dfs[15] = dfs[15].sample(frac=0.1)
dfs[15].head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time_year,date_time_month,date_time_day,...,date_time_dayofweek,date_time_quarter,date_time_is_weekend,date_time_month_sin,date_time_month_cos,date_time_dayofweek_sin,date_time_dayofweek_cos,date_time_hour_sin,date_time_hour_cos,traffic_volume
45196,0,291.35,0.0,0.0,75,Clouds,broken clouds,2018,6,21,...,3,2,False,1.224647e-16,-1.000000,0.433884,-0.900969,0.866025,-0.500000,5490
7641,0,294.22,0.0,0.0,24,Clouds,few clouds,2013,7,13,...,5,3,True,-5.000000e-01,-0.866025,-0.974928,-0.222521,0.866025,-0.500000,2651
44272,0,292.66,0.0,0.0,90,Clouds,overcast clouds,2018,5,21,...,0,2,False,5.000000e-01,-0.866025,0.000000,1.000000,-0.965926,-0.258819,6134
15065,0,293.47,0.0,0.0,12,Clouds,few clouds,2014,6,10,...,1,2,False,1.224647e-16,-1.000000,0.781831,0.623490,-0.500000,0.866025,2388
17792,0,295.55,0.0,0.0,90,Clouds,overcast clouds,2015,8,28,...,4,3,False,-8.660254e-01,-0.500000,-0.433884,-0.900969,-0.965926,-0.258819,5693


### df16
- move the target column to the last place

In [12]:
dfs[16]["BodyFat"] = dfs[16].pop("BodyFat")
dfs[16].head()

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist,BodyFat
0,1.0708,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1,12.3
1,1.0853,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2,6.1
2,1.0414,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6,25.3
3,1.0751,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2,10.4
4,1.0340,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7,28.7


### df19
- remove extra target column

In [13]:
dfs[19] = dfs[19].drop(columns=["verification.result"])
dfs[19].head()

,process.b1.capacity,process.b2.capacity,process.b3.capacity,process.b4.capacity,property.price,property.product,property.winner,verification.time
0,0,0,2,1,59,1,0,163.316667
1,0,0,2,1,59,2,0,200.860000
2,0,0,2,1,59,4,0,154.888889
3,0,0,2,1,59,6,0,108.640000
4,0,0,2,1,60,1,0,85.466667


### df20
- expan date information

In [14]:
dfs[20] = dfs[20].drop(columns=["quarter", "day"])
dfs[20] = extract_date_features(dfs[20], "date")
dfs[20]["actual_productivity"] = dfs[20].pop("actual_productivity") 
dfs[20].head()

,department,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,...,date_month,date_day,date_dayofweek,date_quarter,date_is_weekend,date_month_sin,date_month_cos,date_dayofweek_sin,date_dayofweek_cos,actual_productivity
0,sweing,8,0.80,26.16,1108.0,7080,98,0.0,0,0,...,1,1,3,1,False,0.5,0.866025,0.433884,-0.900969,0.940725
1,finishing,1,0.75,3.94,NaN,960,0,0.0,0,0,...,1,1,3,1,False,0.5,0.866025,0.433884,-0.900969,0.886500
2,sweing,11,0.80,11.41,968.0,3660,50,0.0,0,0,...,1,1,3,1,False,0.5,0.866025,0.433884,-0.900969,0.800570
3,sweing,12,0.80,11.41,968.0,3660,50,0.0,0,0,...,1,1,3,1,False,0.5,0.866025,0.433884,-0.900969,0.800570
4,sweing,6,0.80,25.90,1170.0,1920,50,0.0,0,0,...,1,1,3,1,False,0.5,0.866025,0.433884,-0.900969,0.800382


### df21
- remove `name` column
- move the target column to the last place

In [15]:
dfs[21] = dfs[21].drop(columns=["Name"])
dfs[21]["Combat Power"] = dfs[21].pop("Combat Power")
dfs[21].head()

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Combat Power
0,Grass,Poison,45,49,49,65,65,45,1,False,318
1,Grass,Poison,60,62,63,80,80,60,1,False,405
2,Grass,Poison,80,82,83,100,100,80,1,False,525
3,Fire,NaN,39,52,43,60,50,65,1,False,309
4,Fire,NaN,58,64,58,80,65,80,1,False,405


### df22
- remove `car name` columns
- replace `?` with `NaN`
- move the target column to the last place

In [16]:
dfs[22] = dfs[22].drop(columns=["car name"])
dfs[22] = dfs[22].replace("?", np.nan).astype(float)
dfs[22]["mpg"] = dfs[22].pop("mpg")
dfs[22].head()

,cylinders,displacement,horsepower,weight,acceleration,model year,origin,mpg
0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,18.0
1,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,15.0
2,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,18.0
3,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,16.0
4,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,17.0


### df23
- replace `yes` and `no` with boolean values

In [17]:
dfs[23] = dfs[23].replace({"yes": True, "no": False})
dfs[23].head()

C:\Users\jrsh4\AppData\Local\Temp\ipykernel_11632\868398643.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfs[23] = dfs[23].replace({"yes": True, "no": False})


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,True,southwest,16884.92400
1,18,male,33.770,1,False,southeast,1725.55230
2,28,male,33.000,3,False,southeast,4449.46200
3,33,male,22.705,0,False,northwest,21984.47061
4,32,male,28.880,0,False,northwest,3866.85520


### df24
- move the target column to the last place

In [18]:
dfs[24]["Price"] = dfs[24].pop("Price")
dfs[24].head()

,Company,TypeName,Ram,Weight,TouchScreen,Ips,Ppi,Cpu_brand,HDD,SSD,Gpu_brand,Os,Price
0,Apple,Ultrabook,8,1.37,0,1,226.983005,Intel Core i5,0,128,Intel,Mac,11.175755
1,Apple,Ultrabook,8,1.34,0,0,127.677940,Intel Core i5,0,0,Intel,Mac,10.776777
2,HP,Notebook,8,1.86,0,0,141.211998,Intel Core i5,0,256,Intel,Others,10.329931
3,Apple,Ultrabook,16,1.83,0,1,220.534624,Intel Core i7,0,512,AMD,Mac,11.814476
4,Apple,Ultrabook,8,1.37,0,1,226.983005,Intel Core i5,0,256,Intel,Mac,11.473101


### df26
- remove extra column

In [19]:
dfs[26] = dfs[26].drop(columns=["selector"])
dfs[26].head()

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,85,92,45,27,31,0.0
1,85,64,59,32,23,0.0
2,86,54,33,16,54,0.0
3,91,78,34,24,36,0.0
4,87,70,12,28,10,0.0


### df27
- replace string `month` and `day` with numeric values

In [20]:
month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

day_mapping = {
    'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5, 'sat': 6, 'sun': 7
}

dfs[27]['month'] = dfs[27]['month'].map(month_mapping)
dfs[27]['day'] = dfs[27]['day'].map(day_mapping)
dfs[27].head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


### df31
- remove extra column
- move the target column to the last place

In [21]:
dfs[31] = dfs[31].drop(columns=["Unnamed: 0"])
dfs[31]["hg/ha_yield"] = dfs[31].pop("hg/ha_yield")
dfs[31].head()

,Area,Item,Year,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,hg/ha_yield
0,Albania,Maize,1990,1485.0,121.0,16.37,36613
1,Albania,Potatoes,1990,1485.0,121.0,16.37,66667
2,Albania,"Rice, paddy",1990,1485.0,121.0,16.37,23333
3,Albania,Sorghum,1990,1485.0,121.0,16.37,12500
4,Albania,Soybeans,1990,1485.0,121.0,16.37,7000


## Common changes

In [22]:
def preprocess_dataset(df):
    # Get the target column (the last column in the DataFrame)
    target_column = df.columns[-1]
    print(f"Target column: {target_column}")
    
    print(f"Original shape: {df.shape}")
    
    # Remove duplicates
    df = df.drop_duplicates()
    print(f"After removing duplicates: {df.shape}")
    
    # Remove columns with a single unique value
    single_value_cols = [col for col in df.columns if df[col].nunique() <= 1]
    df = df.drop(columns=single_value_cols)
    if single_value_cols:
        print(f"Dropped columns with single value: {single_value_cols}")
    print(f"After dropping single-value columns: {df.shape}")
    
    # Remove rows with NaN or missing values in the target column
    before_drop = df.shape[0]
    df = df.dropna(subset=[target_column])
    after_drop = df.shape[0]
    print(f"Removed {before_drop - after_drop} rows with missing target values.")
    
    # Splitting columns by type
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
    
    if target_column in numeric_cols:
        numeric_cols.remove(target_column)
    if target_column in categorical_cols:
        categorical_cols.remove(target_column)
    
    binary_categorical_cols = [col for col in categorical_cols if df[col].nunique() == 2]
    non_binary_categorical_cols = [col for col in categorical_cols if col not in binary_categorical_cols]
    
    print(f"Numeric columns: {len(numeric_cols)}")
    print(f"Binary categorical columns: {len(binary_categorical_cols)}")
    print(f"Non-binary categorical columns: {len(non_binary_categorical_cols)}")
    
    # Handle NaNs
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].mean())
    
    for col in categorical_cols:
        df[col] = df[col].fillna('missing')
    
    # One-hot encoding
    if binary_categorical_cols or non_binary_categorical_cols:
        if non_binary_categorical_cols:
            df = pd.get_dummies(df, columns=non_binary_categorical_cols, dummy_na=False)
        
        if binary_categorical_cols:
            df = pd.get_dummies(df, columns=binary_categorical_cols, drop_first=True, dummy_na=False)

        dummy_cols = [col for col in df.columns if col not in numeric_cols + categorical_cols]
        if target_column:
            dummy_cols = [col for col in dummy_cols if col != target_column]
        for col in dummy_cols:
            if df[col].dtype == bool:
                df[col] = df[col].astype(int)
        
        print(f"Shape after one-hot encoding: {df.shape}")
    
    X = df.drop(columns=[target_column])
    y = df[target_column].copy()

    # Standardize numeric features
    numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    if numeric_cols:
        scaler = StandardScaler()
        X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
    
    # Standardize the target column (assuming it's always numeric)
    target_scaler = StandardScaler()
    y_standardized = target_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()
    y = pd.Series(y_standardized, index=y.index, name=target_column)
    
    print(f"Target '{target_column}' standardized - mean: {y.mean():.4f}, std: {y.std():.4f}")
    
    processed_df = pd.concat([X, y], axis=1)

    print(f"Final shape after preprocessing: {processed_df.shape}")
    return processed_df

In [23]:
processed_dfs = {}

for df_idx in dfs:
    print(f"\nProcessing dataset {df_idx}:")
    processed_df = preprocess_dataset(dfs[df_idx])
    processed_dfs[df_idx] = processed_df
    print(f"First 5 rows of processed dataset {df_idx}:")
    print(processed_df.head())


Processing dataset 0:
Target column: ViolentCrimesPerPop
Original shape: (1994, 123)
After removing duplicates: (1994, 123)
After dropping single-value columns: (1994, 123)
Removed 0 rows with missing target values.
Numeric columns: 122
Binary categorical columns: 0
Non-binary categorical columns: 0
Target 'ViolentCrimesPerPop' standardized - mean: -0.0000, std: 1.0003
Final shape after preprocessing: (1994, 123)
First 5 rows of processed dataset 0:
   population  householdsize  racepctblack  racePctWhite  racePctAsian  \
0    1.043612      -0.814997     -0.630002      0.599578     -0.161288   
1   -0.453937      -1.853636     -0.235335     -0.056219      1.418982   
2   -0.453937      -0.265129      1.224931     -0.793990      0.078147   
3   -0.138663       1.873246      3.237730     -2.761379     -0.161288   
4   -0.375118       0.529125     -0.630002      0.804514     -0.304949   

   racePctHisp  agePct12t21  agePct12t29  agePct16t24  agePct65up  ...  \
0     0.111765    -0.54279

## Save data

In [24]:
output_dir = '../data/preprocessed'
os.makedirs(output_dir, exist_ok=True)

for df_idx in processed_dfs:
    output_path = os.path.join(output_dir, f"{df_idx}.csv")
    processed_dfs[df_idx].to_csv(output_path, index=False)
    print(f"Dataset {df_idx} saved to {output_path}")
    print(f"Shape: {processed_dfs[df_idx].shape}")

Dataset 0 saved to ../data/preprocessed\0.csv
Shape: (1994, 123)
Dataset 1 saved to ../data/preprocessed\1.csv
Shape: (506, 14)
Dataset 2 saved to ../data/preprocessed\2.csv
Shape: (17379, 24)
Dataset 3 saved to ../data/preprocessed\3.csv
Shape: (1005, 9)
Dataset 4 saved to ../data/preprocessed\4.csv
Shape: (1020, 46)
Dataset 5 saved to ../data/preprocessed\5.csv
Shape: (10000, 17)
Dataset 6 saved to ../data/preprocessed\6.csv
Shape: (1974, 40)
Dataset 7 saved to ../data/preprocessed\7.csv
Shape: (649, 44)
Dataset 8 saved to ../data/preprocessed\8.csv
Shape: (9527, 5)
Dataset 9 saved to ../data/preprocessed\9.csv
Shape: (5318, 12)
Dataset 10 saved to ../data/preprocessed\10.csv
Shape: (768, 9)
Dataset 11 saved to ../data/preprocessed\11.csv
Shape: (4177, 11)
Dataset 12 saved to ../data/preprocessed\12.csv
Shape: (951, 22)
Dataset 13 saved to ../data/preprocessed\13.csv
Shape: (10000, 21)
Dataset 14 saved to ../data/preprocessed\14.csv
Shape: (1503, 6)
Dataset 15 saved to ../data/prepro